# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [49]:
from azureml.core import Model
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.automl.core.shared import constants
import json as js
import requests as rq
import os
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is from the Kaggle dataset ["HR Analytics: Job Change of Data Scientists"](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists). It lists individuals who have gone through the company's Data Science training program, some demographic and background information on the trainee, and wether or not that employee churned from the company. I will be building an ML pipeline to predict the likelihood of a new candidate churning, with an API accessible to run on an as-needed basis.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
nodes = 6
compute_name = 'cap-compute'

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'cp-experiment'

experiment=Experiment(ws, experiment_name)

In [4]:
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D2_V2',
        #vm_priority = 'lowpriority',
        min_nodes=nodes, max_nodes=nodes,
    )
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

This is a classification task, since the response variable is a yes/no retention indicator. The experiment timeout is set to 20 minutes, as that seems to be a reasonable time to operate on. 6 concurrent iterations because the compute cluster is at 6 nodes max. The blocked models is because of the probmen specified here: https://knowledge.udacity.com/questions/509841

In [5]:
# TODO: Put your automl settings here
automl_settings = dict(
    compute_target=compute_target,
    task="classification",
    training_data=ws.datasets['DsEmpData'],
    label_column_name="target",   
    path='./automl',
    enable_early_stopping=True,
    featurization='auto',
    debug_log="automl_errors.log",
    experiment_timeout_minutes=20,
    max_concurrent_iterations=nodes,
    primary_metric='accuracy',
    blocked_models=['XGBoostClassifier']
)

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-core, training version:1.23.0, current version:1.22.0
Package:azureml-dataprep, training version:2.10.1, current version:2.9.1
Package:azureml-dataprep-native, training version:30.0.0, current version:29.0.0
Package:azureml-dataprep-rslex, training version:1.8.0, current version:1.7.0
Package:azureml-dataset-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-defaults, training version:1.23.0, current version:1.22.0
Package:azureml-interpret, training version:1.23.0, current version:1.22.0
Package:azureml-mlflow, training version:1.23.0, current version:1.22.0
Package:azureml-pipeline-core, training version:1.23.0, current version:1.22.0
Package:azureml-telemetry, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-client, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-runtime, training version:1.23.0, current versio

Run(Experiment: cp-experiment,
Id: AutoML_d6f9ace0-532e-406b-8f0a-be5e73cab558_67,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                             SGDClassifierWrapper(alpha=3.8776122448979593,
                                                                                                  class_weight='balanced',
                  

In [9]:
best_run_metrics = best_run.get_metrics()
print('Best Run Id:', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])

Best Run Id:  AutoML_d6f9ace0-532e-406b-8f0a-be5e73cab558_67

 Accuracy: 0.7989351706858754


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [102]:
os.makedirs('outputs', exist_ok=True)
joblib.dump(fitted_model, filename='outputs/fitted_model.pkl')
automl_model = remote_run.register_model(model_name='DS_HR_Retention_AutoMl')

In [103]:
best_run.download_file('./outputs/scoring_file_v_1_0_0.py', 'score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'envFile.yml')

inference_config = InferenceConfig(entry_script='./score.py', environment=best_run.get_environment())
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=4,
    enable_app_insights=True,
    description='DS HR Retention Model'
)
service = Model.deploy(
    workspace=ws,
    name='ds-hr-retention-model',
    models=[automl_model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True,
)

In [105]:
service.wait_for_deployment(True)
print()
print('/n', service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"

/n Healthy
http://895cffa0-6fa0-404a-bf3e-4e7d77f7b514.southcentralus.azurecontainer.io/score
http://895cffa0-6fa0-404a-bf3e-4e7d77f7b514.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [111]:
test_data = ws.datasets['DsEmpData'].to_pandas_dataframe().sample(2)
test_target = test_data.pop('target')
test_js = {'data': test_data.to_dict(orient='records')}

In [112]:
resp = rq.post(service.scoring_uri, json=test_js)

In [114]:
resp.json()

'{"result": ["0.0", "0.0"]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [100]:
print(service.get_logs())

2021-03-12T22:49:48,421121100+00:00 - iot-server/run 
2021-03-12T22:49:48,431291200+00:00 - nginx/run 
2021-03-12T22:49:48,428152600+00:00 - gunicorn/run 
2021-03-12T22:49:48,481827000+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [101]:
service.delete()

In [ ]:
compute_target.delete()